# Datetime Functions

### Objectives
1. Cast to timestamp
2. Format datetimes
3. Extract from timestamp
4. Convert to date
5. Manipulate datetimes

### Methods
- [Column](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/column.html): `cast`
- [Built-In Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#datetime-functions): `date_format`, `to_date`, `date_add`, `year`, `month`, `dayofweek`, `minute`, `second`


### Generate inline CSV and load it into a DataFrame

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Create sample data
data = {
    "user_id": [1, 2, 3, 4, 5],
    "timestamp": [(datetime.now() - timedelta(days=i)).timestamp() * 1e6 for i in range(5)]  # microseconds
}

# Save to CSV
pdf = pd.DataFrame(data)
csv_path = "/dbfs/tmp/datetimes_events.csv"
pdf.to_csv(csv_path, index=False)

# Load into Spark DataFrame
df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(csv_path))

from pyspark.sql.functions import col
df = df.select("user_id", col("timestamp").alias("timestamp"))
display(df)

### Built-In Functions: Date Time Functions

| Method | Description |
|--------|-------------|
| `add_months` | Returns the date that is numMonths after startDate |
| `current_timestamp` | Returns current timestamp |
| `date_format` | Formats a date/timestamp to a string |
| `dayofweek` | Extracts day of week |
| `from_unixtime` | Converts unix timestamp to datetime |
| `minute` | Extracts minute |
| `unix_timestamp` | Converts time string to unix timestamp |


### Cast to Timestamp using `cast()`

In [ ]:
from pyspark.sql.types import TimestampType

timestamp_df = df.withColumn("timestamp", (col("timestamp") / 1e6).cast(TimestampType()))
display(timestamp_df)

### Format date using `date_format()`

In [ ]:
from pyspark.sql.functions import date_format

formatted_df = (timestamp_df
                .withColumn("date string", date_format("timestamp", "MMMM dd, yyyy"))
                .withColumn("time string", date_format("timestamp", "HH:mm:ss.SSSSSS"))
               )
display(formatted_df)

### Extract datetime attributes

In [ ]:
from pyspark.sql.functions import year, month, dayofweek, minute, second

datetime_df = (timestamp_df
               .withColumn("year", year(col("timestamp")))
               .withColumn("month", month(col("timestamp")))
               .withColumn("dayofweek", dayofweek(col("timestamp")))
               .withColumn("minute", minute(col("timestamp")))
               .withColumn("second", second(col("timestamp")))
              )
display(datetime_df)

### Convert to Date using `to_date()`

In [ ]:
from pyspark.sql.functions import to_date

date_df = timestamp_df.withColumn("date", to_date(col("timestamp")))
display(date_df)

### Add Days using `date_add()`

In [ ]:
from pyspark.sql.functions import date_add

plus_2_df = timestamp_df.withColumn("plus_two_days", date_add(col("timestamp"), 2))
display(plus_2_df)